## List of quality and tidiniess issues in soccer database

### Quality

##### match table
- `date` column should be datetime
- `season` should be categorical
- figure out what `stage` represents
    - represents the matchday
    - ACTION: rename to `matchday` and should be categorical
- player IDs (`home_player_1` ... `away_player_11`) are null for some matches
    - should drop null values
- match events (`goal` ... `possession`) are null for some matches
    - should drop null values
- predictions are null for some matches
    - should drop null values
- player positions (`home_player_X1` ... `away_player_X11`) are null for some matches
    - leave for now as I don't intend to use these values
    - so it's better to have the other values that those rows may provide

##### player attributes table
- `date` column should be datetime
- missing a `season` column
    - create from date column
- preferred foot should be categorical
- all attributes missing for some rows
    - should drop null values
- attacking work rate null for some players
    - should drop null values
- attacking work rate has strange values
    - should normalize values
- defensive work rate has strange values
    - should normalize values
- attacking work rate should be categorical variable
- defensive work rate should be categorical variable

##### team attributes table
- `date` column should be datetime
- missing a `season` column
    - create from date column
- all the columns that end with `Class` should be categorical
- `buildUpPlayDribbling` is float, should be int
    - not handling because will drop the column
- `buildUpPlayDribbling` has (a lot of) null values
    - consider dropping the entire column


##### champs league table
- contains data for years that are not part of our period of interest
    - those years should be removed
- order assigned to `progress` values is not consistent eg `5. Last 16` and `6. Last 16`
    - remove order before making the variables categorical
- `progress` should be categorical
- missing team api id column
    - match from team name and make necessary adjustments
- missing a `season` column
    - create from year column
    - drop rows for years that are not in our timeframe
- missing a league id column
    - create from country column
- inconsistent team names (eg typo in Steaua Bucuresti, shortened names like Bayern Munich and Real Madrid)
    - use names in the main team table
- potential duplicates (APOEL vs Hapoel tel aviv)
    - remove duplicates

### Tidiness

##### match table
- the xml columns (`goal` ... `possession`) contain multiple details that can and should be in a separate table
- player positions (`home_player_X1` ... `away_player_X11`) should be in a separate table
- match players (`home_player_1` ... `away_player_11`) should be in a separate table

##### player attributes table
- duplicated rows for the same information (same player and date)

##### champs league table
- some rows are duplicates

### Iterating - Quality

##### goal events table
- confirm what exactly the `_del` column represents

##### shoton events table
- confirm what exactly the `y` and `_del` columns represent

##### shotoff events table
- confirm what exactly the `y` and `_del` columns represent

### Iterating - Tidiness

##### goal events table